In [6]:
def ps(s):
    """Process String: convert a string into a list of lowercased words."""
    return [word.strip() for word in re.split(r'([+-/*()?]|\s+|\d)', s) if word.strip()]

In [18]:
from pathlib import Path
import re
import numpy as np

# read quesitons and answers from file
dataset_filename = Path("../train_data/arithmetic__mixed.txt")

# q,a lists
questions = []
answers = []

with open(dataset_filename) as dataset_file:
    # Grabbing a subset of the entire file
    for i in range(1000):
        line_q = dataset_file.readline().strip()
        line_a = dataset_file.readline().strip()

        questions.append(ps(line_q))
        answers.append(eval(line_a))
    
# use zip to create dataset object
dataset = [(q,a) for q,a in zip(questions,answers)]

In [19]:
import torch

from fastprogress.fastprogress import progress_bar, master_bar

from random import shuffle

## Preparing data for use as NN input

We can't pass a list of plain text words and tags to a NN. We need to convert them to a more appropriate format.

We'll start by creating a unique index for each word and tag.

In [27]:
word_to_index = {}

total_words = 0

for question, _ in dataset:

    total_words += len(question)

    for word in question:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

index_to_word = dict([(word_to_index[word],word) for word in word_to_index])
            
VOCAB_SIZE = len(word_to_index)

In [21]:
print("       Vocabulary Indices")
print("-------------------------------")

for word in sorted(word_to_index):
    print(f"{word:>14} => {word_to_index[word]:>2}")

print("\nTotal number of words:", total_words)
print("Number of unique words:", len(word_to_index))

       Vocabulary Indices
-------------------------------
             ( =>  3
             ) =>  6
             * => 18
             + =>  2
             - =>  5
             . => 15
             / =>  7
             0 =>  8
             1 =>  0
             2 => 11
             3 => 17
             4 => 10
             5 =>  1
             6 =>  9
             7 =>  4
             8 => 14
             9 => 13
             ? => 21
     Calculate => 12
      Evaluate => 16
          What => 19
            is => 20
            of => 24
           the => 22
         value => 23

Total number of words: 21049
Number of unique words: 25


## Letting the NN parameterize words

Once we have a unique identifier for each word, it is useful to start our NN with an [embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding) layer. This layer converts an index into a vector of values.

You can think of each value as indicating something about the word. For example, maybe the first value indicates how much a word conveys happiness vs sadness. Of course, the NN can learn any attributes and it is not limited to thinks like happy/sad, masculine/feminine, etc.

**Creating an embedding layer**. An embedding layer is created by telling it the size of the vocabulary (the number of words) and an embedding dimension (how many values to use to represent a word).

**Embedding layer input and output**. An embedding layer takes an index and return a matrix.

In [60]:
def encode_onehot(words,mapping):
    encoded = torch.zeros((len(words),VOCAB_SIZE))
    for i in range(len(words)):
        word = words[i]
        encoded[i,mapping[word]] = 1
    return encoded

In [61]:
def decode_onehot(encoded):
    return ''.join(map(str,([index_to_word[idx] for idx in np.where(encoded==1)[1]])))

In [62]:
def convert_to_index_tensor(words, mapping):
    indices = [mapping[w] for w in words]
    return torch.tensor(indices, dtype=torch.long)

In [56]:
vocab_size = len(word_to_index)
embed_dim = 6  # Hyperparameter
embed_layer = torch.nn.Embedding(vocab_size, embed_dim)

In [64]:
# i = torch.tensor([word_to_index["the"], word_to_index["dog"]])
indices = encode_onehot(ps("15 + (7 + -17)"), word_to_index)
#embed_output = embed_layer(indices)
#indices.shape, embed_output.shape, embed_output
indices.shape

torch.Size([10, 25])

## Adding an LSTM layer

The [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) layer is in charge of processing embeddings such that the network can output the correct classification. Since this is a recurrent layer, it will take into account past words when it creates an output for the current word.

**Creating an LSTM layer**. To create an LSTM you need to tell it the size of its input (the size of an embedding) and the size of its internal cell state.

**LSTM layer input and output**. An LSTM takes an embedding (and optionally an initial hidden and cell state) and outputs a value for each word as well as the current hidden and cell state).

If you read the linked LSTM documentation you will see that it requires input in this format: (seq_len, batch, input_size)

As you can see above, our embedding layer outputs something that is (seq_len, input_size). So, we need to add a dimension in the middle.

In [65]:
hidden_dim = 10  # Hyperparameter
num_layers = 5  # Hyperparameter
lstm_layer = torch.nn.LSTM(VOCAB_SIZE, hidden_dim, num_layers=num_layers)

In [67]:
# The LSTM layer expects the input to be in the shape (L, N, E)
#   L is the length of the sequence
#   N is the batch size (we'll stick with 1 here)
#   E is the size of the embedding
lstm_output, _ = lstm_layer(indices.unsqueeze(1))
lstm_output.shape
lstm_output

tensor([[[-0.0091, -0.0424, -0.0885,  0.0592,  0.0317, -0.0574,  0.0056,
          -0.0617,  0.0189, -0.0271]],

        [[-0.0134, -0.0644, -0.1315,  0.0839,  0.0417, -0.0915,  0.0089,
          -0.0883,  0.0328, -0.0321]],

        [[-0.0150, -0.0762, -0.1521,  0.0939,  0.0444, -0.1130,  0.0100,
          -0.0995,  0.0422, -0.0310]],

        [[-0.0155, -0.0829, -0.1620,  0.0976,  0.0448, -0.1268,  0.0100,
          -0.1039,  0.0484, -0.0289]],

        [[-0.0155, -0.0868, -0.1666,  0.0987,  0.0447, -0.1357,  0.0094,
          -0.1053,  0.0524, -0.0272]],

        [[-0.0154, -0.0892, -0.1688,  0.0988,  0.0445, -0.1414,  0.0087,
          -0.1054,  0.0551, -0.0263]],

        [[-0.0153, -0.0908, -0.1698,  0.0985,  0.0443, -0.1449,  0.0081,
          -0.1051,  0.0569, -0.0258]],

        [[-0.0152, -0.0917, -0.1702,  0.0981,  0.0442, -0.1471,  0.0077,
          -0.1048,  0.0580, -0.0257]],

        [[-0.0152, -0.0923, -0.1703,  0.0978,  0.0442, -0.1484,  0.0074,
          -0.1044,  0.0

## Classifiying the LSTM output

We can now add a fully connected, [linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) layer to our NN to learn the correct part of speech (classification).

**Creating a linear layer**. We create a linear layer by specifying the shape of the input into the layer and the number of neurons in the linear layer.

**Linear layer input and output**. The input is expected to be (input_size, output_size) and the output will be the output of each neuron.

In [39]:
linear_layer = torch.nn.Linear(hidden_dim, 1)

In [40]:
linear_output = linear_layer(lstm_output)
linear_output.shape, linear_output

(torch.Size([8, 1, 1]),
 tensor([[[-0.3362]],
 
         [[-0.3672]],
 
         [[-0.3823]],
 
         [[-0.3896]],
 
         [[-0.3932]],
 
         [[-0.3949]],
 
         [[-0.3958]],
 
         [[-0.3963]]], grad_fn=<AddBackward0>))

# Training an LSTM model

In [41]:
# Hyperparameters
valid_percent = 0.2  # Training/validation split

embed_dim = 7  # Size of word embedding
hidden_dim = 8  # Size of LSTM internal state
num_layers = 5  # Number of LSTM layers

learning_rate = 0.1
num_epochs = 2

## Creating training and validation datasets

In [42]:
N = len(dataset)
vocab_size = len(word_to_index)  # Number of unique input words

# Shuffle the data so that we can split the dataset randomly
shuffle(dataset)

split_point = int(N * valid_percent)
valid_dataset = dataset[:split_point]
train_dataset = dataset[split_point:]

len(valid_dataset), len(train_dataset)

(20000, 80000)

## Creating the Parts of Speech LSTM model

In [43]:
class POS_LSTM(torch.nn.Module):
    """Part of Speach LSTM model."""

    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, embed_dim)
        self.lstm = torch.nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers)
        self.linear = torch.nn.Linear(hidden_dim, 1)

    def forward(self, X):
        X = self.embed(X)
        X, _ = self.lstm(X.unsqueeze(1))
        return self.linear(X)

## Training

In [44]:
def compute_accuracy(dataset):
    """A helper function for computing accuracy on the given dataset."""
    total_words = 0
    total_correct = 0

    model.eval()

    with torch.no_grad():
        for sentence, tags in dataset:
            sentence_indices = convert_to_index_tensor(sentence, word_to_index)
            tag_scores = model(sentence_indices).squeeze()
            predictions = tag_scores.argmax(dim=1)
            total_words += len(sentence)
            total_correct += sum(t == tag_list[p] for t, p in zip(tags, predictions))

    return total_correct / total_words

In [45]:
model = POS_LSTM(vocab_size, embed_dim, hidden_dim, num_layers)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

mb = master_bar(range(num_epochs))

# accuracy = compute_accuracy(valid_dataset)
# print(f"Validation accuracy before training : {accuracy * 100:.2f}%")

for epoch in mb:

    # Shuffle the data for each epoch (stochastic gradient descent)
    shuffle(train_dataset)

    model.train()

    for sentence, answer in progress_bar(train_dataset, parent=mb):
        model.zero_grad()
        
        sentence = convert_to_index_tensor(sentence, word_to_index)

        predict = model(sentence)
        
        loss = criterion(predict[-1][0], torch.tensor([answer], dtype=torch.float))
        loss.backward()
        optimizer.step()

#accuracy = compute_accuracy(valid_dataset)
#print(f"Validation accuracy after training : {accuracy * 100:.2f}%")

## Examining results

Here we look at all words that are misclassified by the model

In [ ]:
print("\nMis-predictions after training on entire dataset")
header = "Word".center(14) + " | True Tag | Prediction"
print(header)
print("-" * len(header))

with torch.no_grad():
    for sentence, answer in dataset:
        sentence_indices = convert_to_index_tensor(sentence, word_to_index)
        pred = model(sentence_indices)[-1]
        print(pred, answer)


Mis-predictions after training on entire dataset
     Word      | True Tag | Prediction
--------------------------------------
tensor([[0.2867]]) 5
tensor([[0.2867]]) 7
tensor([[0.2867]]) -6
tensor([[0.2867]]) -6
tensor([[0.2867]]) -3
tensor([[0.2867]]) 0.5
tensor([[0.2867]]) 1
tensor([[0.2867]]) 5
tensor([[0.2867]]) 4
tensor([[0.2867]]) 0
tensor([[0.2867]]) 6
tensor([[0.2867]]) -2
tensor([[0.2867]]) 0.08695652173913043
tensor([[0.2867]]) -5
tensor([[0.2867]]) 0
tensor([[0.2867]]) 5
tensor([[0.2867]]) 5
tensor([[0.2867]]) 8
tensor([[0.2867]]) 0.75
tensor([[0.2867]]) 0.2857142857142857
tensor([[0.2867]]) -0.6666666666666666
tensor([[0.2867]]) -0.6
tensor([[0.2867]]) 1.2
tensor([[0.2867]]) 1
tensor([[0.2867]]) 1
tensor([[0.2867]]) 0.13333333333333333
tensor([[0.2867]]) 6
tensor([[0.2867]]) 0.6666666666666666
tensor([[0.2867]]) 0.6
tensor([[0.2867]]) -0.4
tensor([[0.2867]]) 5
tensor([[0.2867]]) 11
tensor([[0.2867]]) -7
tensor([[0.2867]]) -0.5714285714285714
tensor([[0.2867]]) -6
tensor([

tensor([[0.2867]]) -0.2
tensor([[0.2867]]) -0.3333333333333333
tensor([[0.2867]]) 0.13333333333333333
tensor([[0.2867]]) 0.11764705882352941
tensor([[0.2867]]) 3
tensor([[0.2867]]) -5
tensor([[0.2867]]) -8
tensor([[0.2867]]) 0.25
tensor([[0.2867]]) 2
tensor([[0.2867]]) -1
tensor([[0.2867]]) 0.5714285714285714
tensor([[0.2867]]) -0.07142857142857142
tensor([[0.2867]]) -0.42857142857142855
tensor([[0.2867]]) 3
tensor([[0.2867]]) 3
tensor([[0.2867]]) -0.25
tensor([[0.2867]]) -0.18181818181818182
tensor([[0.2867]]) -2
tensor([[0.2867]]) -4
tensor([[0.2867]]) -4
tensor([[0.2867]]) 1.5
tensor([[0.2867]]) -2
tensor([[0.2867]]) 0.15384615384615385
tensor([[0.2867]]) 2
tensor([[0.2867]]) -2.5
tensor([[0.2867]]) 1
tensor([[0.2867]]) -5
tensor([[0.2867]]) -0.2222222222222222
tensor([[0.2867]]) 0.18181818181818182
tensor([[0.2867]]) 0.3333333333333333
tensor([[0.2867]]) -0.16666666666666666
tensor([[0.2867]]) -5
tensor([[0.2867]]) -0.42857142857142855
tensor([[0.2867]]) -2
tensor([[0.2867]]) -1
te

tensor([[0.2867]]) 0.5
tensor([[0.2867]]) 0.6
tensor([[0.2867]]) 0.2857142857142857
tensor([[0.2867]]) 0.06451612903225806
tensor([[0.2867]]) 0.2222222222222222
tensor([[0.2867]]) 0
tensor([[0.2867]]) 3
tensor([[0.2867]]) 1.3333333333333333
tensor([[0.2867]]) -4
tensor([[0.2867]]) 0
tensor([[0.2867]]) -0.10526315789473684
tensor([[0.2867]]) -0.6666666666666666
tensor([[0.2867]]) -0.25
tensor([[0.2867]]) 9
tensor([[0.2867]]) -0.5
tensor([[0.2867]]) -3
tensor([[0.2867]]) 5
tensor([[0.2867]]) 4
tensor([[0.2867]]) -0.75
tensor([[0.2867]]) -2
tensor([[0.2867]]) 2.5
tensor([[0.2867]]) -0.2
tensor([[0.2867]]) 5
tensor([[0.2867]]) 0.375
tensor([[0.2867]]) -1
tensor([[0.2867]]) 0.5
tensor([[0.2867]]) -0.10526315789473684
tensor([[0.2867]]) -0.4
tensor([[0.2867]]) 2
tensor([[0.2867]]) -9
tensor([[0.2867]]) -5
tensor([[0.2867]]) 6
tensor([[0.2867]]) -0.2857142857142857
tensor([[0.2867]]) 0.13333333333333333
tensor([[0.2867]]) -0.8
tensor([[0.2867]]) 5
tensor([[0.2867]]) 7
tensor([[0.2867]]) -4
te

tensor([[0.2867]]) -0.3333333333333333
tensor([[0.2867]]) -0.42857142857142855
tensor([[0.2867]]) 0
tensor([[0.2867]]) 0.3333333333333333
tensor([[0.2867]]) 3
tensor([[0.2867]]) -3
tensor([[0.2867]]) -5
tensor([[0.2867]]) 1
tensor([[0.2867]]) 0.2857142857142857
tensor([[0.2867]]) 4
tensor([[0.2867]]) 0.8333333333333334
tensor([[0.2867]]) 0.15384615384615385
tensor([[0.2867]]) -0.3333333333333333
tensor([[0.2867]]) 0.4444444444444444
tensor([[0.2867]]) 2.5
tensor([[0.2867]]) -2
tensor([[0.2867]]) -4
tensor([[0.2867]]) -0.125
tensor([[0.2867]]) -2
tensor([[0.2867]]) 0.4
tensor([[0.2867]]) -0.2222222222222222
tensor([[0.2867]]) 1
tensor([[0.2867]]) -2
tensor([[0.2867]]) 2
tensor([[0.2867]]) 0.8333333333333334
tensor([[0.2867]]) 3
tensor([[0.2867]]) 0.3333333333333333
tensor([[0.2867]]) -7
tensor([[0.2867]]) -1
tensor([[0.2867]]) 0.5
tensor([[0.2867]]) -0.6666666666666666
tensor([[0.2867]]) 0.6666666666666666
tensor([[0.2867]]) -0.2
tensor([[0.2867]]) 0.3333333333333333
tensor([[0.2867]]) 

tensor([[0.2867]]) -2.5
tensor([[0.2867]]) -0.5454545454545454
tensor([[0.2867]]) 2
tensor([[0.2867]]) -0.2857142857142857
tensor([[0.2867]]) 0
tensor([[0.2867]]) -0.3333333333333333
tensor([[0.2867]]) -6
tensor([[0.2867]]) -2
tensor([[0.2867]]) -0.4
tensor([[0.2867]]) -0.25
tensor([[0.2867]]) 8
tensor([[0.2867]]) -0.75
tensor([[0.2867]]) -0.2857142857142857
tensor([[0.2867]]) 4
tensor([[0.2867]]) 6
tensor([[0.2867]]) -0.16666666666666666
tensor([[0.2867]]) 0.1
tensor([[0.2867]]) -0.42857142857142855
tensor([[0.2867]]) 0.2222222222222222
tensor([[0.2867]]) -0.4
tensor([[0.2867]]) 0.6
tensor([[0.2867]]) -2
tensor([[0.2867]]) -8
tensor([[0.2867]]) -5
tensor([[0.2867]]) -0.4
tensor([[0.2867]]) -3
tensor([[0.2867]]) -6
tensor([[0.2867]]) -1.25
tensor([[0.2867]]) 1.3333333333333333
tensor([[0.2867]]) 3
tensor([[0.2867]]) 0.14285714285714285
tensor([[0.2867]]) 0.2222222222222222
tensor([[0.2867]]) 0.4444444444444444
tensor([[0.2867]]) -0.5
tensor([[0.2867]]) -0.5454545454545454
tensor([[0.28

...

tensor([[0.2867]]) 0.75
tensor([[0.2867]]) 0.5
tensor([[0.2867]]) -3
tensor([[0.2867]]) 0.42857142857142855
tensor([[0.2867]]) 0.4
tensor([[0.2867]]) 0.5454545454545454
tensor([[0.2867]]) 6
tensor([[0.2867]]) -5
tensor([[0.2867]]) -0.5
tensor([[0.2867]]) 4
tensor([[0.2867]]) -0.2222222222222222
tensor([[0.2867]]) 0.25
tensor([[0.2867]]) 1.6666666666666667
tensor([[0.2867]]) 5
tensor([[0.2867]]) 3
tensor([[0.2867]]) 7
tensor([[0.2867]]) -0.15384615384615385
tensor([[0.2867]]) -1
tensor([[0.2867]]) 0.2857142857142857
tensor([[0.2867]]) -9
tensor([[0.2867]]) 5
tensor([[0.2867]]) -8
tensor([[0.2867]]) -0.2857142857142857
tensor([[0.2867]]) 0.08695652173913043
tensor([[0.2867]]) 0.14285714285714285
tensor([[0.2867]]) -3
tensor([[0.2867]]) -0.25
tensor([[0.2867]]) -0.14285714285714285
tensor([[0.2867]]) -2
tensor([[0.2867]]) -0.2857142857142857
tensor([[0.2867]]) -0.3333333333333333
tensor([[0.2867]]) 0.75
tensor([[0.2867]]) 2
tensor([[0.2867]]) -4
tensor([[0.2867]]) -2
tensor([[0.2867]]) 0.

tensor([[0.2867]]) -0.08695652173913043
tensor([[0.2867]]) -0.5
tensor([[0.2867]]) 0.25
tensor([[0.2867]]) -2
tensor([[0.2867]]) 0
tensor([[0.2867]]) 0.6666666666666666
tensor([[0.2867]]) 0.8
tensor([[0.2867]]) -0.16666666666666666
tensor([[0.2867]]) 4
tensor([[0.2867]]) 4
tensor([[0.2867]]) -2
tensor([[0.2867]]) 0.18181818181818182
tensor([[0.2867]]) 0.058823529411764705
tensor([[0.2867]]) 7
tensor([[0.2867]]) 8
tensor([[0.2867]]) 0
tensor([[0.2867]]) -1
tensor([[0.2867]]) 5
tensor([[0.2867]]) 0.5714285714285714
tensor([[0.2867]]) -11
tensor([[0.2867]]) 0.4
tensor([[0.2867]]) -5
tensor([[0.2867]]) -5
tensor([[0.2867]]) -0.75
tensor([[0.2867]]) 0.6
tensor([[0.2867]]) -6
tensor([[0.2867]]) -4
tensor([[0.2867]]) 0.15384615384615385
tensor([[0.2867]]) 0.375
tensor([[0.2867]]) 2
tensor([[0.2867]]) -0.2222222222222222
tensor([[0.2867]]) -1.5
tensor([[0.2867]]) -5
tensor([[0.2867]]) 4
tensor([[0.2867]]) -0.11764705882352941
tensor([[0.2867]]) -3
tensor([[0.2867]]) -0.8571428571428571
tensor(

tensor([[0.2867]]) -4
tensor([[0.2867]]) 5
tensor([[0.2867]]) 0
tensor([[0.2867]]) -3
tensor([[0.2867]]) -4.5
tensor([[0.2867]]) -1.5
tensor([[0.2867]]) 5
tensor([[0.2867]]) 1.5
tensor([[0.2867]]) 0.5
tensor([[0.2867]]) 0.2222222222222222
tensor([[0.2867]]) 1.5
tensor([[0.2867]]) 4
tensor([[0.2867]]) 4
tensor([[0.2867]]) -0.3333333333333333
tensor([[0.2867]]) 1
tensor([[0.2867]]) 4
tensor([[0.2867]]) -5
tensor([[0.2867]]) 6
tensor([[0.2867]]) 4
tensor([[0.2867]]) 0.2
tensor([[0.2867]]) 0.6
tensor([[0.2867]]) -0.125
tensor([[0.2867]]) 3
tensor([[0.2867]]) -4
tensor([[0.2867]]) -1
tensor([[0.2867]]) 0.5714285714285714
tensor([[0.2867]]) -4
tensor([[0.2867]]) 0.6666666666666666
tensor([[0.2867]]) -3
tensor([[0.2867]]) 1.3333333333333333
tensor([[0.2867]]) 0.18181818181818182
tensor([[0.2867]]) -2
tensor([[0.2867]]) -0.2222222222222222
tensor([[0.2867]]) 0.8333333333333334
tensor([[0.2867]]) 5
tensor([[0.2867]]) 0.2222222222222222
tensor([[0.2867]]) 2
tensor([[0.2867]]) 0
tensor([[0.2867]]

tensor([[0.2867]]) 0
tensor([[0.2867]]) 1.2
tensor([[0.2867]]) 1
tensor([[0.2867]]) 0
tensor([[0.2867]]) -4
tensor([[0.2867]]) -5
tensor([[0.2867]]) 1.5
tensor([[0.2867]]) -6
tensor([[0.2867]]) -4
tensor([[0.2867]]) -4
tensor([[0.2867]]) -0.13333333333333333
tensor([[0.2867]]) 0.5714285714285714
tensor([[0.2867]]) -1
tensor([[0.2867]]) -4
tensor([[0.2867]]) -3
tensor([[0.2867]]) -7
tensor([[0.2867]]) 1
tensor([[0.2867]]) -5
tensor([[0.2867]]) 0
tensor([[0.2867]]) 0
tensor([[0.2867]]) 0.2222222222222222
tensor([[0.2867]]) 0.25
tensor([[0.2867]]) -0.42857142857142855
tensor([[0.2867]]) -0.5
tensor([[0.2867]]) 0.75
tensor([[0.2867]]) -0.5714285714285714
tensor([[0.2867]]) 0.5
tensor([[0.2867]]) 3
tensor([[0.2867]]) -2
tensor([[0.2867]]) -1
tensor([[0.2867]]) -2.5
tensor([[0.2867]]) -0.5
tensor([[0.2867]]) 0.2857142857142857
tensor([[0.2867]]) -0.6666666666666666
tensor([[0.2867]]) 0.14285714285714285
tensor([[0.2867]]) 5
tensor([[0.2867]]) -1
tensor([[0.2867]]) 0.25
tensor([[0.2867]]) -3


tensor([[0.2867]]) 0.07407407407407407
tensor([[0.2867]]) -7
tensor([[0.2867]]) 0.18181818181818182
tensor([[0.2867]]) -4
tensor([[0.2867]]) -3
tensor([[0.2867]]) 0.3333333333333333
tensor([[0.2867]]) -0.8
tensor([[0.2867]]) 0.4
tensor([[0.2867]]) -9
tensor([[0.2867]]) 4
tensor([[0.2867]]) 0.125
tensor([[0.2867]]) -0.2222222222222222
tensor([[0.2867]]) -0.75
tensor([[0.2867]]) 4
tensor([[0.2867]]) -6
tensor([[0.2867]]) -0.16666666666666666
tensor([[0.2867]]) 5
tensor([[0.2867]]) -3
tensor([[0.2867]]) 2
tensor([[0.2867]]) -0.375
tensor([[0.2867]]) 2
tensor([[0.2867]]) 2
tensor([[0.2867]]) 1
tensor([[0.2867]]) 4
tensor([[0.2867]]) 1.5
tensor([[0.2867]]) -0.16666666666666666
tensor([[0.2867]]) 8
tensor([[0.2867]]) 7
tensor([[0.2867]]) 0.2
tensor([[0.2867]]) -0.18181818181818182
tensor([[0.2867]]) -6
tensor([[0.2867]]) 4
tensor([[0.2867]]) -0.3333333333333333
tensor([[0.2867]]) -9
tensor([[0.2867]]) -0.1111111111111111
tensor([[0.2867]]) -7
tensor([[0.2867]]) 6
tensor([[0.2867]]) -0.181818

## Using the model for inference

In [ ]:
new_sentence = "3 + 3"

# Convert sentence to lowercase words
sentence = ps(new_sentence)

# Check that each word is in our vocabulary
for word in sentence:
    assert word in word_to_index

# Convert input to a tensor
sentence = convert_to_index_tensor(sentence, word_to_index)

# Compute prediction
predictions = model(sentence)
predictions = predictions.squeeze().argmax(dim=1)

# Print results
for word, tag in zip(ps(new_sentence), predictions):
    print(word, "=>", tag_list[tag.item()])

Things to try:

- compare with fully connected network
- compare with CNN
- compare with transformer